In [26]:
# lead_scout_pipeline.py  (단일 파일)
import os, json, requests, datetime, pytz
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from pprint import pprint

# ───────────────────────────────────────
# 0. 설정
# ───────────────────────────────────────
load_dotenv()
OPENAI_MODEL = "gpt-4.1-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# ───────────────────────────────────────
# 1. 상태 정의
# ───────────────────────────────────────
class LeadScoutState(TypedDict):
    company_info: Dict[str, Any]                           # 회사 요약
    planner_response:   Annotated[List, add_messages]      # 키워드·필터
    generator_response: Annotated[List, add_messages]      # 10개 리드 (1·2번 필수 정보)
    tavily_response:    Annotated[List, add_messages]      # 최신 기사 보강
    reporter_response:  Annotated[List, add_messages]      # 최종 스키마 배열
    final_report:       Annotated[List, add_messages]      # status 래퍼 JSON

latest = lambda st, k: st[k][-1] if st[k] else None

# ───────────────────────────────────────
# 2. 노드
# ───────────────────────────────────────
## 2‑1 Planner ─ 키워드·필터 추출
planner_prompt = """
<역할> B2B 세일즈 타게팅 플래너
<목표> 입력된 회사 요약으로부터
      ① keywords(4-6개), ② exclusion, ③ location
을 JSON 배열 형태로 작성하세요.
반드시 JSON만 출력.
"""
def planner_node(state: LeadScoutState):
    summary = state["company_info"]["summary"]
    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    resp = llm.invoke([{"role":"system","content": planner_prompt + "\n<요약>\n"+summary}])
    try: plan = json.loads(resp.content)
    except: plan = {"keywords":[],"exclusion":[],"location":"대한민국"}
    state["planner_response"].append(HumanMessage(content=json.dumps(plan,ensure_ascii=False)))
    return state

## 2‑2 Generator ─ 필수정보 10개 리드
def generator_node(state: LeadScoutState):
    c      = state["company_info"]
    plan   = json.loads(latest(state,"planner_response").content)
    kws    = ", ".join(plan["keywords"])
    llm    = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    prompt = f"""
<역할> 세일즈 전문가
<키워드> {kws}
대한민국 소재, 경쟁사 제외. 아래 스키마로 리드 10개를 JSON 배열로 작성.
[
 {{"companyName":"","industryKeywords":[],"homepage":"","location":"",
   "founderOrCEO":"","founded":"","summary":"","targetCustomer":""}}
]
"""
    resp = llm.invoke([{"role":"system","content":prompt}])
    try: leads = json.loads(resp.content)
    except: leads = []
    state["generator_response"].append(HumanMessage(content=json.dumps(leads,ensure_ascii=False)))
    return state

## 2‑3 Enricher ─ 각 리드 최신 기사 1건
def enricher_node(state: LeadScoutState):
    leads = json.loads(latest(state,"generator_response").content)
    headers = {"Authorization":f"Bearer {TAVILY_API_KEY}","Content-Type":"application/json"}
    enrich=[]
    for ld in leads:
        q=f"{ld['companyName']} 최근 뉴스"
        try:
            r=requests.post("https://api.tavily.com/search",headers=headers,json={"query":q,"max_results":1})
            if r.status_code==200 and r.json().get("results"):
                item=r.json()["results"][0]
                enrich.append({"companyName":ld["companyName"],
                                "title":item.get("title",""),
                                "snippet":item.get("content","")[:120],
                                "url":item.get("url","")})
        except: pass
    state["tavily_response"].append(HumanMessage(content=json.dumps(enrich,ensure_ascii=False)))
    return state

# 2‑4 Reporter ─ 모든 필드 보강 + leadScore
def reporter_node(state: LeadScoutState):
    # ① 기본 리드 + 보강 맵
    base = json.loads(latest(state, "generator_response").content)
    tavily_raw = latest(state, "tavily_response")
    enrich_map = {}
    if tavily_raw:                 # 보강 정보가 있을 때만 생성
        enrich_map = {e["companyName"]: e
                        for e in json.loads(tavily_raw.content)}

    # ② 두 정보 merge
    merged = []
    for ld in base:
        e = enrich_map.get(ld["companyName"], {})
        merged.append({**ld,
            "financials": {}, "latestTrends": [e.get("snippet", "")] if e else [],
            "competitors": [], "strengths": [], "risks": [],
            "leadScore": 0,
            "referenceLinks": [{"title": e.get("title", ""), "url": e.get("url", "")}] if e else []
        })

    # ③ LLM으로 summary·score·etc 보강 추후 score -> 알고리즘 구현
    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
    prompt = (
        "<역할> B2B 세일즈 분석가. 아래 JSON 배열에 "
        "summary, financials, competitors, strengths, risks, leadScore(0‑10)를 채워 반환.\n"
        "반드시 JSON 배열만 출력.\n"
        + json.dumps(merged, ensure_ascii=False)
    )
    resp = llm.invoke([{"role": "system", "content": prompt}])
    try:
        final_leads = json.loads(resp.content)
    except json.JSONDecodeError:
        final_leads = merged  # LLM 실패 시 fallback

    msg = HumanMessage(content=json.dumps(final_leads, ensure_ascii=False))
    state["reporter_response"].append(msg)

    # 👉 delta 로 reporter_response 만 반환
    return {"reporter_response": state["reporter_response"]}


# ───────────────────────────────────────
# 3. 그래프
# ───────────────────────────────────────
def run():
    g = StateGraph(LeadScoutState)
    g.add_node("planner",   planner_node)
    g.add_node("generator", generator_node)
    g.add_node("enricher",  enricher_node)
    g.add_node("reporter",  reporter_node)

    g.set_entry_point("planner")
    g.add_edge("planner",   "generator")
    g.add_edge("generator", "enricher")
    g.add_edge("enricher",  "reporter")
    g.set_finish_point("reporter")

    app = g.compile()

    init_state: LeadScoutState = {
        "company_info": {"summary": company_summary},
        "planner_response":   [],
        "generator_response": [],
        "tavily_response":    [],
        "reporter_response":  []
    }

    for delta in app.stream(init_state):
        for key, value in delta.items():
            pprint(f"노드 '{key}' 실행됨")
            # Reporter 단계에서 최종 JSON 추출
            if key == "reporter" and "reporter_response" in value:
                final_json_str = value["reporter_response"][-1].content
                print("\n--- Lead List JSON ---\n")
                pprint(final_json_str)
        pprint("\n---\n")

if __name__ == "__main__":
    run()


"노드 'planner' 실행됨"
'\n---\n'
"노드 'generator' 실행됨"
'\n---\n'
"노드 'enricher' 실행됨"
'\n---\n'
"노드 'reporter' 실행됨"

--- Lead List JSON ---

('[{"companyName": "에코그린솔루션", "summary": "에코그린솔루션은 친환경 재생에너지 분야에서 태양광 발전 시스템을 '
 '개발 및 설치하는 기업으로, 중소기업 및 공공기관을 주요 고객으로 삼고 있습니다. SK디앤디의 인적 분할을 통해 에너지 전문회사인 '
 '에코그린이 신설되는 등 에너지 산업 내에서 주목받고 있습니다.", "financials": {"revenue": "정보 없음", '
 '"profit": "정보 없음", "funding": "정보 없음"}, "competitors": ["태양광 발전 시스템 개발 및 설치 '
 '업체", "친환경 에너지 솔루션 기업"], "strengths": ["친환경 재생에너지 전문성", "태양광 발전 시스템 개발 및 설치 '
 '경험", "중소기업 및 공공기관 대상 맞춤형 솔루션 제공"], "risks": ["재생에너지 시장 경쟁 심화", "정부 정책 및 보조금 '
 '변화에 따른 영향", "기술 발전 속도에 따른 대응 필요"], "leadScore": 7}, {"companyName": '
 '"헬스케어인사이트", "summary": "헬스케어인사이트는 의료 데이터 분석과 맞춤형 헬스케어 솔루션을 제공하는 스타트업으로, 병원 및 '
 '건강관리 기관을 주요 고객으로 합니다. 약국 판매 데이터 분석 등 다양한 헬스케어 데이터 활용에 집중하고 있습니다.", '
 '"financials": {"revenue": "정보 없음", "profit": "정보 없음", "funding": "정보 없음"}, '
 '"competitors": ["의료 데이터 분석 기업", "헬스케어 솔루션 제공 업체"], "strengths": ["의료 데이터 분석 '
 '전문성", "맞

In [24]:
company_summary = '''
이 회사는 기업을 위한 SaaS(Software as a Service) 솔루션을 제공하는 IT 기업으로, 주로 경영 지원 시스템의 비효율성을 해결하는 데 중점을 두고 있습니다. 회사의 주요 제품과 서비스는 다음과 같습니다:

### 문제점
1. **비효율적인 경영 지원 시스템**: 많은 기업들이 여전히 수작업으로 반복적인 업무를 처리하고 있으며, 이는 전체 업무 시간의 40% 이상을 차지합니다.
2. **커뮤니케이션 오버헤드**: 팀 간의 비효율적인 커뮤니케이션으로 인해 추가적인 시간과 비용이 발생합니다.
3. **자산 관리의 어려움**: 유형 자산 및 무형 자산의 관리가 비효율적으로 이루어지고 있습니다.

### 솔루션
1. **SaaS 기반 경영 지원 시스템**: 
   - **자동화**: 반복적인 업무를 자동화하여 효율성을 높입니다.
   - **AI 에이전트**: AI 기반의 에이전트를 통해 실시간으로 데이터를 분석하고 인사이트를 제공합니다.
   - **자산 관리**: QR 코드 기반의 자산 관리 시스템을 통해 자산의 효율적인 관리가 가능합니다.

2. **커뮤니케이션 효율화**:
   - **통합 커뮤니케이션 플랫폼**: 다양한 커뮤니케이션 도구를 통합하여 오버헤드를 줄입니다.
   - **데이터 기반 인사이트**: 커뮤니케이션 데이터를 분석하여 효율성을 높이는 전략을 제안합니다.

3. **데이터 인사이트 제공**:
   - **실시간 데이터 분석**: 기업 데이터를 실시간으로 분석하여 경영 인사이트를 제공합니다.
   - **리스크 및 유효 기간 관리**: 자산의 유효 기간을 관리하고 만료 알림을 제공합니다.

### 비즈니스 모델
- **서비스 제공**: 국내 기업에 SaaS 및 솔루션을 제공하며, 임직원 서비스도 함께 제공합니다.
- **수익 모델**: 월 사용료 및 라이선스 비용을 통해 수익을 창출합니다. 다양한 플랜(베이직, 비즈니스, 엔터프라이즈)을 통해 고객의 필요에 맞춘 서비스를 제공합니다.

### 시장 및 목표
- **시장 확장**: 글로벌 빅데이터 및 분석 솔루션 시장으로의 확장을 목표로 하고 있습니다.
- **매출 목표**: 2030년까지 매출 11000억 원 달성을 목표로 하고 있습니다.

### 팀 구성
- **CEO**: 유민재
- **CTO**: 김종민
- **개발자**: 이승현

이 회사는 IT 자산 관리 및 데이터 분석 솔루션을 통해 기업의 경영 효율성을 높이고, 글로벌 시장으로의 확장을 목표로 하고 있습니다.
'''

NameError: name 'state' is not defined